In [1]:
import pymysql
conn = pymysql.connect(host='localhost',port=3306,db='lolgamequiz',user='root',passwd='root',charset='utf8')
def sele(sql):
    cur = conn.cursor()
    cur.execute(sql)
    result = list(cur.fetchall())
    result = [list(r) for r in result]
    conn.commit()
    cur.close()
    
    return result


In [2]:
import numpy as np
import pandas as pd
from collections import defaultdict

In [95]:
sql = 'select date,processor, dayNum, dayOrder, teamA, teamB, scoreA, scoreB, fullTeamA, fullTeamB from world2019'
data = sele(sql)
results = pd.DataFrame(data, columns=['date', 'processor','dayNum','dayOrder', 'teamA', 'teamB', 'scoreA', 'scoreB', 'fullTeamA', 'fullTeamB'])
print(results)

sql = 'select teamName, rank, region from rank2019 where region <> "Play-In"'
data = sele(sql)
rank2019 = pd.DataFrame(data, columns=['teamName', 'rank', 'region'])
print(rank2019)

          date   processor         dayNum dayOrder teamA teamB scoreA scoreB  \
0   2019-10-02     Play-In          Day 1        1    CG   UOL      0      1   
1   2019-10-02     Play-In          Day 1        2   DFM   SPY      0      1   
2   2019-10-02     Play-In          Day 1        3   MMM   UOL      1      0   
3   2019-10-02     Play-In          Day 1        4   DFM   ISG      0      1   
4   2019-10-02     Play-In          Day 1        5   MMM    CG      0      1   
..         ...         ...            ...      ...   ...   ...    ...    ...   
83  2019-10-27  Main_Event  Quarterfinals        3   SKT   SPY      3      1   
84  2019-10-27  Main_Event  Quarterfinals        4   DWG    G2      1      3   
85  2019-11-02  Main_Event     Semifinals        2    iG   FPX      1      3   
86  2019-11-03  Main_Event     Semifinals        1    G2   SKT      3      1   
87  2019-11-10  Main_Event         Finals        1    G2   FPX      0      3   

             fullTeamA             full

In [96]:
# clean data
# cover data type
results = results.astype({'date':'datetime64', 'scoreA':'int64', 'scoreB':'int64', 'fullTeamA':'string', 'fullTeamB':'string'})
# fix team name
results['teamA'] = results['teamA'].apply(lambda x: x.upper())
results['teamB'] = results['teamB'].apply(lambda x: x.upper())
results['fullTeamA'] = results['fullTeamA'].apply(lambda x: x.upper())
results['fullTeamB'] = results['fullTeamB'].apply(lambda x: x.upper())
# sort
results = results.sort_values(by=['date','dayNum','dayOrder'])
results = results.reset_index()
results = results.drop(['index'], axis=1)
print(results)

####################
# rank2019
# rank
rank2019 = rank2019.astype({'rank':'int64', 'teamName':'string'})
rank2019['teamName'] = rank2019['teamName'].apply(lambda x: x.upper())
#results = results.merge(rank2019, how='left', left_on='fullTeamA', right_on='teamName')
#rank2019
print(rank2019)

         date   processor         dayNum dayOrder teamA teamB  scoreA  scoreB  \
0  2019-10-02     Play-In          Day 1        1    CG   UOL       0       1   
1  2019-10-02     Play-In          Day 1        2   DFM   SPY       0       1   
2  2019-10-02     Play-In          Day 1        3   MMM   UOL       1       0   
3  2019-10-02     Play-In          Day 1        4   DFM   ISG       0       1   
4  2019-10-02     Play-In          Day 1        5   MMM    CG       0       1   
..        ...         ...            ...      ...   ...   ...     ...     ...   
83 2019-10-27  Main_Event  Quarterfinals        3   SKT   SPY       3       1   
84 2019-10-27  Main_Event  Quarterfinals        4   DWG    G2       1       3   
85 2019-11-02  Main_Event     Semifinals        2    IG   FPX       1       3   
86 2019-11-03  Main_Event     Semifinals        1    G2   SKT       3       1   
87 2019-11-10  Main_Event         Finals        1    G2   FPX       0       3   

             fullTeamA     

In [97]:
df = results['scoreA'].append(results['scoreB'])
df = pd.DataFrame(df)
df.columns = ['score']
gy = df['score'].groupby(df['score'])
print(gy.count())
print(results[results['scoreA']==results['scoreB']])

score
0    79
1    85
2     1
3    11
Name: score, dtype: int64
Empty DataFrame
Columns: [date, processor, dayNum, dayOrder, teamA, teamB, scoreA, scoreB, fullTeamA, fullTeamB]
Index: []


In [98]:
# won
results['Awon'] = False
results['Bwon'] = False
results['Awon'] = results['scoreA']>results['scoreB']
results['Bwon'] = results['scoreA']<results['scoreB']
results

,date,processor,dayNum,dayOrder,teamA,teamB,scoreA,scoreB,fullTeamA,fullTeamB,Awon,Bwon
0,2019-10-02,Play-In,Day 1,1,CG,UOL,0,1,CLUTCH GAMING,UNICORNS OF LOVE.CIS,False,True
1,2019-10-02,Play-In,Day 1,2,DFM,SPY,0,1,DETONATION FOCUSME,SPLYCE,False,True
2,2019-10-02,Play-In,Day 1,3,MMM,UOL,1,0,MAMMOTH,UNICORNS OF LOVE.CIS,True,False
3,2019-10-02,Play-In,Day 1,4,DFM,ISG,0,1,DETONATION FOCUSME,ISURUS,False,True
4,2019-10-02,Play-In,Day 1,5,MMM,CG,0,1,MAMMOTH,CLUTCH GAMING,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...
83,2019-10-27,Main_Event,Quarterfinals,3,SKT,SPY,3,1,SK TELECOM T1,SPLYCE,True,False
84,2019-10-27,Main_Event,Quarterfinals,4,DWG,G2,1,3,DAMWON GAMING,G2 ESPORTS,False,True
85,2019-11-02,Main_Event,Semifinals,2,IG,FPX,1,3,INVICTUS GAMING,FUNPLUS PHOENIX,False,True
86,2019-11-03,Main_Event,Semifinals,1,G2,SKT,3,1,G2 ESPORTS,SK TELECOM T1,True,False


In [99]:
"""
############################################
# region
results['regionA'] = None
results['regionB'] = None
results['AisLCK'] = False
results['AisLCS'] = False
results['AisLPL'] = False
results['AisLEC'] = False
results['BisLCK'] = False
results['BisLCS'] = False
results['BisLPL'] = False
results['BisLEC'] = False
for index, row in results.iterrows():
    teamA = row['fullTeamA']
    teamB = row['fullTeamB']
    regionA = rank2019[rank2019['teamName']==teamA]
    if not regionA.empty:
        regionA = regionA['region'].values[0]
        results.loc[index,'regionA'] = regionA
        if regionA == 'LCK':
            results.loc[index,'AisLCK'] = True
        elif regionA == 'LCS':
            results.loc[index,'AisLCS'] = True
        elif regionA == 'LPL':
            results.loc[index,'AisLPL'] = True
        elif regionA == 'LEC':
            results.loc[index,'AisLEC'] = True
    
    regionB = rank2019[rank2019['teamName']==teamB]
    if not regionB.empty:
        regionB = regionB['region'].values[0]
        results.loc[index,'regionB'] = regionB
        if regionB == 'LCK':
            results.loc[index,'BisLCK'] = True
        elif regionB == 'LCS':
            results.loc[index,'BisLCS'] = True
        elif regionB == 'LPL':
            results.loc[index,'BisLPL'] = True
        elif regionB == 'LEC':
            results.loc[index,'BisLEC'] = True

############################################
# rank
results['rankA'] = 5
results['rankB'] = 5
results['Ahigh'] = False
results['Bhigh'] = False
for index, row in results.iterrows():
    teamA = row['fullTeamA']
    teamB = row['fullTeamB']
    rankA = rank2019[rank2019['teamName']==teamA]
    if not rankA.empty:
        rankA = rankA['rank'].values[0]
    else:
        rankA = 5

    rankB = rank2019[rank2019['teamName']==teamB]
    if not rankB.empty:
        rankB = rankB['rank'].values[0]
    else:
        rankB = 5
    
    if rankA < rankB:
        results.loc[index,'Ahigh'] = True
    if rankB < rankA:
        results.loc[index,'Bhigh'] = True
        
    results.loc[index,'rankA'] = rankA
    results.loc[index,'rankB'] = rankB
    

##################################
# last won
results['Alastwon'] = False
results['Blastwon'] = False
won_last = defaultdict(bool)
for index, row in results.iterrows():
    teamA = row['teamA']
    teamB = row['teamB']
    if index == 0:
        won_last[teamA] = row['Awon']
        won_last[teamB] = row['Bwon']
        continue

    results.loc[index, 'Alastwon'] = won_last[teamA]
    results.loc[index, 'Blastwon'] = won_last[teamB]
    won_last[teamA] = row['Awon']
    won_last[teamB] = row['Bwon']

##################################
# last who won
results['lastAwon'] = False
results['lastBwon'] = False
won_last = defaultdict(bool)
for index, row in results.iterrows():
    teamA = row['teamA']
    teamB = row['teamB']
    if index == 0:
        continue
    for index1, row1 in results[:index].iterrows():
        if row1['teamA'] == teamA and row1['teamB'] == teamB:
            results.loc[index, 'lastAwon'] = results.loc[index1, 'Awon']
            results.loc[index, 'lastBwon'] = results.loc[index1, 'Bwon']
        elif row1['teamA'] == teamB and row1['teamB'] == teamA:
            results.loc[index, 'lastAwon'] = results.loc[index1, 'Bwon']
            results.loc[index, 'lastBwon'] = results.loc[index1, 'Awon']

##################################
# BO
def bo(score, win_num):
    max_score = max(score['scoreA'], score['scoreB'])
    if max_score == win_num:
        return True
    else:
        return False
        
results['isBo1'] = results[['scoreA','scoreB']].apply(bo, args=(1,), axis=1)
results['isBo5'] = results[['scoreA','scoreB']].apply(bo, args=(3,), axis=1)

results
"""


"\n############################################\n# region\nresults['regionA'] = None\nresults['regionB'] = None\nresults['AisLCK'] = False\nresults['AisLCS'] = False\nresults['AisLPL'] = False\nresults['AisLEC'] = False\nresults['BisLCK'] = False\nresults['BisLCS'] = False\nresults['BisLPL'] = False\nresults['BisLEC'] = False\nfor index, row in results.iterrows():\n    teamA = row['fullTeamA']\n    teamB = row['fullTeamB']\n    regionA = rank2019[rank2019['teamName']==teamA]\n    if not regionA.empty:\n        regionA = regionA['region'].values[0]\n        results.loc[index,'regionA'] = regionA\n        if regionA == 'LCK':\n            results.loc[index,'AisLCK'] = True\n        elif regionA == 'LCS':\n            results.loc[index,'AisLCS'] = True\n        elif regionA == 'LPL':\n            results.loc[index,'AisLPL'] = True\n        elif regionA == 'LEC':\n            results.loc[index,'AisLEC'] = True\n    \n    regionB = rank2019[rank2019['teamName']==teamB]\n    if not regionB.e

In [100]:

############################################
# region/rank
results['regionA'] = ''
results['regionB'] = ''
results['rankA'] = ''
results['rankB'] = ''

for index, row in results.iterrows():
    teamA = row['fullTeamA']
    teamB = row['fullTeamB']
    regionA = rank2019.loc[rank2019[rank2019['teamName']==teamA].index,'region']
    regionB = rank2019.loc[rank2019[rank2019['teamName']==teamB].index,'region']
    
    rankA = rank2019.loc[rank2019[rank2019['teamName']==teamA].index,'rank']
    rankB = rank2019.loc[rank2019[rank2019['teamName']==teamB].index,'rank']
    
    if not regionA.empty:
        results.loc[index, 'regionA'] = regionA.values[0]
        results.loc[index, 'rankA'] = rankA.values[0]
    if not regionB.empty:
        results.loc[index, 'regionB'] = regionB.values[0]
        results.loc[index, 'rankB'] = rankB.values[0]

##################################
# last won
results['Alastwon'] = False
results['Blastwon'] = False
won_last = defaultdict(bool)
for index, row in results.iterrows():
    teamA = row['teamA']
    teamB = row['teamB']
    if index == 0:
        won_last[teamA] = row['Awon']
        won_last[teamB] = row['Bwon']
        continue

    results.loc[index, 'Alastwon'] = won_last[teamA]
    results.loc[index, 'Blastwon'] = won_last[teamB]
    won_last[teamA] = row['Awon']
    won_last[teamB] = row['Bwon']

##################################
# last who won
results['lastAwon'] = False
results['lastBwon'] = False
won_last = defaultdict(bool)
for index, row in results.iterrows():
    teamA = row['teamA']
    teamB = row['teamB']
    if index == 0:
        continue
    for index1, row1 in results[:index].iterrows():
        if row1['teamA'] == teamA and row1['teamB'] == teamB:
            results.loc[index, 'lastAwon'] = results.loc[index1, 'Awon']
            results.loc[index, 'lastBwon'] = results.loc[index1, 'Bwon']
        elif row1['teamA'] == teamB and row1['teamB'] == teamA:
            results.loc[index, 'lastAwon'] = results.loc[index1, 'Bwon']
            results.loc[index, 'lastBwon'] = results.loc[index1, 'Awon']

##################################
# BO
def bo(score):
    max_score = max(score['scoreA'], score['scoreB'])
    if max_score == 1:
        return True
    elif max_score == 3:
        return False
        
results['BO1'] = results[['scoreA','scoreB']].apply(bo, axis=1)

results



,date,processor,dayNum,dayOrder,teamA,teamB,scoreA,scoreB,fullTeamA,fullTeamB,...,Bwon,regionA,regionB,rankA,rankB,Alastwon,Blastwon,lastAwon,lastBwon,BO1
0,2019-10-02,Play-In,Day 1,1,CG,UOL,0,1,CLUTCH GAMING,UNICORNS OF LOVE.CIS,...,True,North America,CIS,3,1,False,False,False,False,True
1,2019-10-02,Play-In,Day 1,2,DFM,SPY,0,1,DETONATION FOCUSME,SPLYCE,...,True,Japan,Europe,1,3,False,False,False,False,True
2,2019-10-02,Play-In,Day 1,3,MMM,UOL,1,0,MAMMOTH,UNICORNS OF LOVE.CIS,...,False,Oceania,CIS,1,1,False,True,False,False,True
3,2019-10-02,Play-In,Day 1,4,DFM,ISG,0,1,DETONATION FOCUSME,ISURUS,...,True,Japan,Latin America,1,1,False,False,False,False,True
4,2019-10-02,Play-In,Day 1,5,MMM,CG,0,1,MAMMOTH,CLUTCH GAMING,...,True,Oceania,North America,1,3,True,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,2019-10-27,Main_Event,Quarterfinals,3,SKT,SPY,3,1,SK TELECOM T1,SPLYCE,...,False,Korea,Europe,1,3,True,False,False,False,False
84,2019-10-27,Main_Event,Quarterfinals,4,DWG,G2,1,3,DAMWON GAMING,G2 ESPORTS,...,True,Korea,Europe,3,1,True,False,False,False,False
85,2019-11-02,Main_Event,Semifinals,2,IG,FPX,1,3,INVICTUS GAMING,FUNPLUS PHOENIX,...,True,China,China,3,1,True,True,False,False,False
86,2019-11-03,Main_Event,Semifinals,1,G2,SKT,3,1,G2 ESPORTS,SK TELECOM T1,...,False,Europe,Korea,1,1,True,True,False,False,False


In [18]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
#X_previouswins = results[['AisLCK','AisLCS','AisLPL','AisLEC','BisLCK','BisLCS','BisLPL','BisLEC','rankA','rankB','lastAwon','lastBwon','isBo1','isBo5']].values

from sklearn.preprocessing import OneHotEncoder
encoder=OneHotEncoder()
for col in data.columns:
    data[col] = labelencoder.fit_transform(data[col])
 
data.head()
 
y = data['class']    #用列名访问更直观



X_previouswins = results[['regionA','regionB','rankA','rankB','lastAwon','lastBwon','BO1']].values

y_true = results[['Awon']].values
clf = DecisionTreeClassifier(random_state=14)
scores = cross_val_score(clf, X_previouswins, y_true, scoring='accuracy')
print('score: {:.2f}%'.format(np.mean(scores)*100))


score: nan%


F:\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

  FitFailedWarning)
F:\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

  FitFailedWarning)
F:\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

  FitFailedWarning)
F:\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:

In [156]:
from sklearn.preprocessing import LabelEncoder
encoding = LabelEncoder()
encoding.fit_transform(results['regionA'].values)
regionA = encoding.fit_transform(results['regionA'].values)
regionB = encoding.fit_transform(results['regionB'].values)

encoding.fit_transform(results['dayNum'].values)
dayNum = encoding.fit_transform(results['dayNum'].values)

region = np.vstack([regionA, regionB,dayNum]).T

from sklearn.preprocessing import OneHotEncoder
onehot = OneHotEncoder()
r = onehot.fit_transform(region).todense()


df = pd.DataFrame(r)



In [157]:
X_previouswins = results[['rankA','rankB']]
X_previouswins = results[['lastAwon','lastBwon','BO1']]
X_previouswins = results[[]]
#X_previouswins = results[['regionA','regionB','rankA','rankB','lastAwon','lastBwon','Alastwon','Blastwon','BO1']].values

X_previouswins = X_previouswins.join(df)
X_previouswins = X_previouswins.values
clf = DecisionTreeClassifier(random_state=14)
scores = cross_val_score(clf, X_previouswins, y_true, scoring='accuracy')
print('score: {:.2f}%'.format(np.mean(scores)*100))




score: 68.24%


In [144]:


from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(random_state=14)
scores = cross_val_score(clf, X_previouswins, y_true, scoring='accuracy')
print('score: {:.2f}%'.format(np.mean(scores)*100))

F:\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
F:\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
F:\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
F:\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please ch

score: 58.04%


F:\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


In [135]:
from sklearn.model_selection import GridSearchCV
parameter_space = {
                   "max_features": ['auto'],
                   "n_estimators": [100,],
                   "criterion": ["gini", "entropy"],
                   "min_samples_leaf": [2, 4, 6],
                   }
clf = RandomForestClassifier(random_state=14)
grid = GridSearchCV(clf, parameter_space)
grid.fit(X_previouswins, y_true)
print("Accuracy: {0:.1f}%".format(grid.best_score_ * 100))
print(grid.best_estimator_)

F:\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
F:\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
F:\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
F:\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please ch

Accuracy: 72.9%
RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=2, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=14, verbose=0,
                       warm_start=False)


F:\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:739: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


In [136]:
forest = RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=2, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=14, verbose=0,
                       warm_start=False)

scores = cross_val_score(forest, X_previouswins, y_true, scoring='accuracy')
print('score: {:.2f}%'.format(np.mean(scores)*100))
forest.fit(X_previouswins,y_true)

from sklearn.externals import joblib
joblib.dump(forest, 'forest.m')


F:\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
F:\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
F:\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
F:\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please ch

score: 72.88%


['forest.m']